#Assignment 2: Implementation of TF-IDF
## Dreamy Pujara - 202211005



In [1]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("ILSUM/ILSUM-1.0","English")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [4]:
data = dataset["train"]
data

Dataset({
    features: ['id', 'Article', 'Heading', 'Summary'],
    num_rows: 12565
})

In [5]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

In [6]:
df=pd.DataFrame(data)

In [7]:
dataset = Dataset.from_pandas(df, split='train')

In [10]:
df.describe

<bound method NDFrame.describe of                                      id  ...                                            Summary
0      3938f547c863630032649c54e611e6b0  ...  Since the invasion of Ukraine, the value of th...
1      1b336d62e9502c5b91b7afd3c7bcff46  ...  India on Wednesday banned the pro-Khalistan Si...
2      082262f1c5a9e1112c39369403ff4e9a  ...  Kishida replaces outgoing party leader Prime M...
3      82cce3b894007faf93e567d751dd2aef  ...  Pakistan Sikh Council (PSC) Patron-in-Chief Sa...
4      4b3bbf58136527fff41adc6c2f37d73d  ...  The Police said the attacker, Usman Khan, was ...
...                                 ...  ...                                                ...
12560  1f4dc11a672137f6f6cc33c9c1644040  ...  Tariffs of 10 per cent and 5 per cent will tak...
12561  963b9f0a8d88948857626afb9f185d2c  ...  Iran warned Tuesday that new U.S. sanctions ta...
12562  0c52632fe53e8671377c4873f64c2957  ...  A previous clinical trial reported similar res...
12563 

### Perform preprocessing steps on the ”Heading” field of the dataset i.e. lowercase, stopword removal, stemming

In [11]:
from datasets import load_dataset
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string


In [14]:
import nltk

# Download the NLTK stopwords resource
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
def preprocess_text(text):
    # Lowercase conversion
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove stopwords and perform stemming
    tokens = text.split()
    filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]

    # Join the filtered tokens back into a string
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

In [22]:
# Apply preprocessing to the "Heading" field of the dataset
preprocessed_headings = [preprocess_text(heading) for heading in dataset['Heading']]

# Print a few preprocessed headings for demonstration
for i in range(5):
    print(f"Original: {dataset['Heading'][i]}\nPreprocessed: {preprocessed_headings[i]}\n")



Original: Russia Ukraine War: Mastercard, Visa suspend operations in Russia after invasion
Preprocessed: russia ukrain war mastercard visa suspend oper russia invas

Original: Author-Diplomat Vikas Swarup moved to Delhi as Indo-Canadian ties remain cold
Preprocessed: authordiplomat vika swarup move delhi indocanadian tie remain cold

Original: Japan’s ex-top diplomat Fumio Kishida to become new Prime Minister
Preprocessed: japan’ extop diplomat fumio kishida becom new prime minist

Original: Pakistan Sikhs urge India to facilitate access to Dera Baba Nanak
Preprocessed: pakistan sikh urg india facilit access dera baba nanak

Original: London knife attack suspect was jailed for 6 years in 2012 on terror charges: UK police
Preprocessed: london knife attack suspect jail 6 year 2012 terror charg uk polic



### 2. Implement TF-IDF from scratch:

####1. Calculate term frequency of the words
####2.Calculate inverse document frequency of the words
####3.Form the term-document matrix and display the shape of the matrix





In [32]:
import numpy as np
from collections import Counter  # Import Counter from collections
from math import log


In [25]:
# Combine all headings into a single list
headings = dataset['Heading']
all_headings = ' '.join(headings)



In [26]:
# Tokenize the text (split into words)
words = all_headings.split()



In [33]:
# Calculate term frequency (TF) for each word
def calculate_tf(document):
    word_counts = Counter(document)
    total_words = len(document)
    tf = {word: count / total_words for word, count in word_counts.items()}
    return tf

In [28]:
# Calculate inverse document frequency (IDF) for each word
def calculate_idf(corpus, word):
    num_documents_with_word = sum(1 for doc in corpus if word in doc)
    total_documents = len(corpus)
    idf = log(total_documents / (1 + num_documents_with_word))
    return idf


In [29]:
# Calculate TF-IDF for each word in the corpus
def calculate_tfidf(corpus):
    tfidf = []
    for doc in corpus:
        tf = calculate_tf(doc)
        doc_tfidf = {word: tf[word] * calculate_idf(corpus, word) for word in doc}
        tfidf.append(doc_tfidf)
    return tfidf

In [30]:
# Tokenize each heading and store them in a list
tokenized_headings = [heading.split() for heading in headings]

In [34]:
# Calculate TF-IDF for the tokenized headings
tfidf_scores = calculate_tfidf(tokenized_headings)

In [35]:
# Create a list of unique words (vocabulary)
vocabulary = set(word for doc in tokenized_headings for word in doc)

In [36]:
# Form the term-document matrix
term_document_matrix = np.zeros((len(headings), len(vocabulary)))
for doc_index, doc in enumerate(tokenized_headings):
    for word_index, word in enumerate(vocabulary):
        term_document_matrix[doc_index, word_index] = tfidf_scores[doc_index].get(word, 0)


In [37]:
# Display the shape of the term-document matrix
print("Shape of Term-Document Matrix:", term_document_matrix.shape)

Shape of Term-Document Matrix: (12565, 22288)


### 3. Use TF-IDF vectorizer from sklearn library to generate TF-IDF matrix.

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import load_dataset

In [39]:
# Extract the headings from the dataset
headings = dataset['Heading']


In [40]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()



In [41]:
# Fit and transform the headings to generate the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(headings)



### 4. Display the shape of the TF-IDF matrix.

In [42]:
# Display the shape of the TF-IDF matrix
print("Shape of TF-IDF Matrix:", tfidf_matrix.shape)

Shape of TF-IDF Matrix: (12565, 12985)
